In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifer

In [2]:
tf.reset_default_graph()

<h1>Hyperparameter</h1>

In [7]:
epochs = 10
batch_size = 128
learning_rate=0.001

<h1>Placeholder</h1>

In [4]:
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(None, 10), name='output_y')

<h1>LeNet-5</h1>

In [9]:
def LeNet_5(x):

    # Layer 1 : Convolutional Layer. Input = 32x32x3, Output = 28x28x3.
    conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,3,6],mean = 0, stddev = 0.1))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b 
    # TODO: Activation.
    conv1 = tf.nn.relu(conv1)

    # Pooling Layer. Input = 28x28x3. Output = 14x14x6.
    pool_1 = tf.nn.max_pool(conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')


    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = 0, stddev = 0.1))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b
    # TODO: Activation.
    conv2 = tf.nn.relu(conv2)
    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')


    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc1 = tf.contrib.layers.flatten(pool_2)


    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = 0, stddev = 0.1))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc1,fc1_w) + fc1_b

    # TODO: Activation.
    fc1 = tf.nn.relu(fc1)

    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = 0, stddev = 0.1))
    fc2_b = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(fc1,fc2_w) + fc2_b
    # TODO: Activation.
    fc2 = tf.nn.relu(fc2)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_w = tf.Variable(tf.truncated_normal(shape = (84,10), mean = 0 , stddev = 0.1))
    fc3_b = tf.Variable(tf.zeros(10))
    logits = tf.matmul(fc2, fc3_w) + fc3_b
    return logits

<h1>Cost and Optimization</h1>

In [10]:
logits = LeNet_5(x)

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [15]:
def train_neural_network(session, optimizer, feature_batch, label_batch):
    session.run(optimizer, 
                feed_dict={
                    x: feature_batch,
                    y: label_batch
                })

In [16]:
def print_stats(session, feature_batch, label_batch, valid_features, valid_labels, cost, accuracy):
    loss = sess.run(cost, 
                    feed_dict={
                        x: feature_batch,
                        y: label_batch
                    })
    valid_acc = sess.run(accuracy, 
                         feed_dict={
                             x: valid_features,
                             y: valid_labels
                         })
    
    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss, valid_acc))

In [17]:
features_train=pickle.load(open('features_train.pkl','rb'))
features_valid=pickle.load(open('features_valid.pkl','rb'))
features_test=pickle.load(open('features_test.pkl','rb'))
labels_train=pickle.load(open('labels_train.pkl','rb'))
labels_valid=pickle.load(open('labels_valid.pkl','rb'))
labels_test=pickle.load(open('labels_test.pkl','rb'))

In [19]:
sess=tf.Session()
# Initializing the variables
sess.run(tf.global_variables_initializer())

# Training cycle
for epoch in range(epochs):
    # Loop over all batches
    n_batches = 5
    for i in range(n_batches):
        train_neural_network(sess, optimizer, features_train[i], labels_train[i].toarray())

        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, i+1), end='')
        print_stats(sess, features_train[i], labels_train[i].toarray(), features_valid[i], labels_valid[i].toarray(), cost, accuracy)

Epoch  1, CIFAR-10 Batch 1:  Loss:     2.2981 Validation Accuracy: 0.101125
Epoch  1, CIFAR-10 Batch 2:  Loss:     2.2989 Validation Accuracy: 0.109250
Epoch  1, CIFAR-10 Batch 3:  Loss:     2.2959 Validation Accuracy: 0.121000
Epoch  1, CIFAR-10 Batch 4:  Loss:     2.2956 Validation Accuracy: 0.120000
Epoch  1, CIFAR-10 Batch 5:  Loss:     2.2939 Validation Accuracy: 0.123125
Epoch  2, CIFAR-10 Batch 1:  Loss:     2.2878 Validation Accuracy: 0.130375
Epoch  2, CIFAR-10 Batch 2:  Loss:     2.2896 Validation Accuracy: 0.125125
Epoch  2, CIFAR-10 Batch 3:  Loss:     2.2838 Validation Accuracy: 0.141125
Epoch  2, CIFAR-10 Batch 4:  Loss:     2.2842 Validation Accuracy: 0.126375
Epoch  2, CIFAR-10 Batch 5:  Loss:     2.2799 Validation Accuracy: 0.133625
Epoch  3, CIFAR-10 Batch 1:  Loss:     2.2737 Validation Accuracy: 0.139250
Epoch  3, CIFAR-10 Batch 2:  Loss:     2.2784 Validation Accuracy: 0.133625
Epoch  3, CIFAR-10 Batch 3:  Loss:     2.2696 Validation Accuracy: 0.149125
Epoch  3, CI

In [22]:
result=sess.run(accuracy,feed_dict={x: features_test,y: labels_test.toarray()})
print(result)

0.1622
